In [1]:
import os
import timeit
from time import time
import sys
from typing import List

import numpy as np
import openai
from dotenv import load_dotenv

from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

notebook_path = os.path.abspath(get_ipython().starting_dir)
parent_path = os.path.dirname(notebook_path)

sys.path.append(parent_path)
from attribution.experiment_logger import ExperimentLogger
from attribution.token_perturbation import (
    get_replacement_token,
    get_most_similar_token_ids,
)
from attribution.attribution_metrics import (
    cosine_similarity_attribution,
    token_prob_difference,
    token_displacement,
)

/Users/sebastian/Projects/llm-attribution/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
model = GPT2LMHeadModel.from_pretrained("gpt2")  # or any other checkpoint
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

word_token_embeddings = model.transformer.wte.weight.detach().numpy()
position_embeddings = model.transformer.wpe.weight.detach().numpy()
token_cosine_distances = None

logger = ExperimentLogger()

In [13]:
def get_model_output(input: str) -> openai.types.chat.chat_completion.Choice:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": input}],
        temperature=0.0,
        seed=0,
        logprobs=True,
        top_logprobs=20,
    )
    return response.choices[0]


def calculate_token_importance_in_sequence(
    input_sequence: str,
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizer,
    perturbation_strategy: str = "fixed",
    attribution_strategies: List[str] = [
        "cosine",
        "prob_diff",
        "token_displacement",
    ],
    logger: ExperimentLogger = None,
):
    timestamp = time()
    tokens = tokenizer.tokenize(input_sequence)
    token_ids = tokenizer.encode(input_sequence, add_special_tokens=False)
    original_output = get_model_output(input_sequence)
    print(f"Chat Completion - Original: {round(time() - timestamp, 2)}s")

    if logger:
        logger.start_experiment(
            input_sequence, original_output.message.content, perturbation_strategy
        )

    exp_timestamp = time()
    words = input_sequence.split()
    token_index = 0
    for word in words:
        word_tokens = tokenizer.tokenize(word)
        word_token_ids = tokenizer.encode(word, add_special_tokens=False)
        start_word_time = time()
        replacement_token_ids = [
            get_replacement_token(
                word_token_ids[i],
                perturbation_strategy,
                word_token_embeddings,
                tokenizer,
            )
            for i in range(len(word_tokens))
        ]
        print(
            f"\nReplaced word '{word}': {round(time() - start_word_time, 2)}s - get_replacement_token()"
        )

        # Replace the current word with the new tokens
        perturbed_input = tokenizer.decode(
            token_ids[:token_index]
            + replacement_token_ids
            + token_ids[token_index + len(word_tokens) :]
        )

        # Get the output logprobs for the perturbed input
        timestamp = time()
        perturbed_output = get_model_output(perturbed_input)
        print(f"Chat Completion - Perturbed: {round(time() - timestamp, 2)}s")

        timestamp = time()
        for attribution_strategy in attribution_strategies:
            attributed_tokens = tokens
            if attribution_strategy == "cosine":
                cosine_timestamp = time()
                sentence_attr, token_attributions = cosine_similarity_attribution(
                    original_output, perturbed_output, model, tokenizer
                )
                cosine_timestamp_end = time()
            elif attribution_strategy == "prob_diff":
                prob_diff_timestamp = time()
                sentence_attr, attributed_tokens, token_attributions = (
                    token_prob_difference(
                        original_output.logprobs, perturbed_output.logprobs
                    )
                )
                prob_diff_timestamp_end = time()
            elif attribution_strategy == "token_displacement":
                token_displacement_timestamp = time()
                sentence_attr, attributed_tokens, token_attributions = (
                    token_displacement(
                        original_output.logprobs, perturbed_output.logprobs
                    )
                )
                token_displacement_timestamp_end = time()
            else:
                raise ValueError(
                    f"Unknown attribution strategy: {attribution_strategy}"
                )

            if logger:
                start_logging = time()
                for i in range(len(word_tokens)):
                    logger.log_input_token_attribution(
                        attribution_strategy,
                        token_index + i,
                        word_tokens[i],
                        float(sentence_attr),
                    )
                    for j, attr_score in enumerate(token_attributions):
                        logger.log_token_attribution_matrix(
                            attribution_strategy,
                            token_index + i,
                            j,
                            attributed_tokens[j],
                            attr_score.squeeze(),
                        )
                end_logging = time()
        time_all_attrs = time() - timestamp
        print(f"Attributions computation: {time_all_attrs}s")
        print(f"- Cosine Attr: {round(cosine_timestamp_end - cosine_timestamp, 2)}s")
        print(
            f"- Prob Diff Attr: {round(prob_diff_timestamp_end - prob_diff_timestamp, 2)}s"
        )
        print(
            f"- Token Displacement Attr: {round(token_displacement_timestamp_end - token_displacement_timestamp, 2)}s"
        )
        print(f"- Attr Logging: {round(end_logging - start_logging, 2)}s")
        print(f"Total for word '{word}': {round(time() - start_word_time, 2)}s")

        token_index += len(word_tokens)

    print(f"\n\nExp Total: {time() - exp_timestamp}s\n\n")

    if logger:
        logger.log_perturbation(
            i,
            tokenizer.decode(replacement_token_ids)[0],
            perturbation_strategy,
            input_sequence,
            original_output.message.content,
            perturbed_input,
            perturbed_output.message.content,
        )
        logger.stop_experiment()

    return (original_output.message.content,)

In [14]:
input_sequences = [
    "The clock shows 9:47 PM. What time does the clock show?",
    "The building is 132 meters tall. How tall is the building?",
    "The package weighs 8.6 kilograms. How much does the package weigh?",
    "The thermometer reads 23 degrees Celsius. What is the temperature according to the thermometer?",
    "She drove 157 kilometers to visit her friend. How far did she drive to visit her friend?",
    "John has 83 books on his shelf. How many books does John have on his shelf?",
    "Maria is 37 years old today. How old is Maria?",
    "There are 68 people registered for the webinar. How many people are registered for the webinar?",
    "Alex saved $363 from his birthday gifts. How much money did Alex save?",
    "The recipe requires 14 teaspoons of sugar. How many teaspoons of sugar does the recipe require?",
]


for input_sequence in input_sequences:
    for perturbation_strategy in ["distant"]:
        original_output = calculate_token_importance_in_sequence(
            input_sequence,
            model,
            tokenizer,
            perturbation_strategy,
            attribution_strategies=["cosine", "prob_diff", "token_displacement"],
            logger=logger,
        )

        print(
            input_sequence,
            original_output,
        )

Chat Completion - Original: 1.52s

Replaced word 'The': 0.24s - get_replacement_token()
Chat Completion - Perturbed: 1.06s
Attributions computation: 0.03852391242980957s
- Cosine Attr: 0.0s
- Prob Diff Attr: 0.0s
- Token Displacement Attr: 0.0s
- Attr Logging: 0.01s
Total for word 'The': 1.34s

Replaced word 'clock': 0.28s - get_replacement_token()
Chat Completion - Perturbed: 0.92s
Attributions computation: 0.03969001770019531s
- Cosine Attr: 0.0s
- Prob Diff Attr: 0.0s
- Token Displacement Attr: 0.0s
- Attr Logging: 0.01s
Total for word 'clock': 1.24s

Replaced word 'shows': 0.23s - get_replacement_token()
Chat Completion - Perturbed: 0.98s
Attributions computation: 0.03743720054626465s
- Cosine Attr: 0.0s
- Prob Diff Attr: 0.0s
- Token Displacement Attr: 0.0s
- Attr Logging: 0.01s
Total for word 'shows': 1.25s

Replaced word '9:47': 0.72s - get_replacement_token()
Chat Completion - Perturbed: 1.51s
Attributions computation: 0.13027501106262207s
- Cosine Attr: 0.0s
- Prob Diff Attr: 

In [20]:
display(logger.df_experiments)

,exp_id,original_input,original_output,perturbation_strategy,duration
0,1,The clock shows 9:47 PM. What time does the cl...,The clock shows 9:47 PM.,distant,15.893994
1,2,The clock shows 9:47 PM. What time does the cl...,The clock shows 9:47 PM.,distant,16.084582
2,3,The building is 132 meters tall. How tall is t...,The building is 132 meters tall.,distant,21.002312
3,4,The package weighs 8.6 kilograms. How much doe...,The package weighs 8.6 kilograms.,distant,20.078763
4,5,The thermometer reads 23 degrees Celsius. What...,The temperature according to the thermometer i...,distant,23.73309
5,6,She drove 157 kilometers to visit her friend. ...,She drove 157 kilometers to visit her friend.,distant,26.523367
6,7,John has 83 books on his shelf. How many books...,John has 83 books on his shelf.,distant,26.890062
7,8,Maria is 37 years old today. How old is Maria?,Maria is 37 years old.,distant,16.271812
8,9,There are 68 people registered for the webinar...,There are 68 people registered for the webinar.,distant,29.417536
9,10,Alex saved $363 from his birthday gifts. How m...,Alex saved $363.,distant,19.571603


In [21]:
logger.print_sentence_attribution()

,exp_id,attribution_strategy,perturbation_strategy,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17,token_18,token_19,token_20,token_21,token_22,token_23,token_24,token_25,token_26
0,1,cosine,distant,The 0.00,clock 0.00,shows 0.00,9 0.02,: 0.02,47 0.02,PM 0.02,. 0.02,What 0.00,time 0.04,does 0.11,the 0.00,clock 0.00,show 0.05,? 0.05,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1,prob_diff,distant,The 0.03,clock 0.02,shows 0.02,9 0.24,: 0.24,47 0.24,PM 0.23,. 0.23,What 0.05,time 0.34,does 0.93,the 0.08,clock 0.03,show 0.78,? 0.78,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,1,token_displacement,distant,The 0.00,clock 0.00,shows 0.00,9 1.11,: 1.11,47 1.11,PM 2.67,. 2.67,What 0.00,time 2.56,does 16.78,the 0.00,clock 0.00,show 11.44,? 11.44,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2,cosine,distant,The 0.00,clock 0.00,shows 0.00,9 0.02,: 0.02,47 0.02,PM 0.02,. 0.02,What 0.00,time 0.04,does 0.11,the 0.00,clock 0.00,show 0.05,? 0.05,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2,prob_diff,distant,The 0.01,clock 0.02,shows 0.02,9 0.24,: 0.24,47 0.24,PM 0.24,. 0.24,What 0.06,time 0.34,does 0.92,the 0.10,clock 0.04,show 0.78,? 0.78,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,2,token_displacement,distant,The 0.00,clock 0.00,shows 0.00,9 1.00,: 1.00,47 1.00,PM 3.00,. 3.00,What 0.00,time 2.44,does 16.67,the 0.00,clock 0.00,show 11.11,? 11.11,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,3,cosine,distant,The 0.00,building 0.11,is 0.00,132 0.16,met 0.14,ers 0.14,tall 0.00,. 0.00,How 0.00,tall 0.00,is 0.00,the 0.15,building 0.17,? 0.17,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,3,prob_diff,distant,The 0.00,building 0.97,is 0.00,132 1.00,met 1.00,ers 1.00,tall 0.01,. 0.01,How 0.08,tall 0.01,is 0.09,the 0.90,building 1.00,? 1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,3,token_displacement,distant,The 0.00,building 17.62,is 0.00,132 17.25,met 17.38,ers 17.38,tall 0.00,. 0.00,How 0.00,tall 0.00,is 0.00,the 9.00,building 17.50,? 17.50,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,4,cosine,distant,The 0.00,package 0.00,we 0.03,igh 0.03,s 0.03,8 0.05,. 0.05,6 0.05,kil 0.06,ograms 0.06,. 0.06,How 0.00,much 0.20,does 0.14,the 0.00,package 0.00,we 0.00,igh 0.00,? 0.00,nan,nan,nan,nan,nan,nan,nan


In [22]:
logger.print_attribution_matrix(exp_id=1)

Attribution matrix for cosine with perturbation strategy distant:
Input Tokens (Rows) vs. Output Tokens (Columns)


,The (0),clock (1),shows (2),9 (3),: (4),47 (5),PM (6),. (7)
The (0),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
clock (1),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
shows (2),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
9 (3),0.000000,-0.000000,-0.000000,0.289177,0.000000,0.274936,0.000000,0.000000
: (4),0.000000,-0.000000,-0.000000,0.289177,0.000000,0.274936,0.000000,0.000000
47 (5),0.000000,-0.000000,-0.000000,0.289177,0.000000,0.274936,0.000000,0.000000
PM (6),0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.821500,1.000000
. (7),0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.821500,1.000000
What (8),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
time (9),0.000000,-0.000000,0.000000,0.488054,0.000000,-0.000000,0.821500,1.000000


Attribution matrix for prob_diff with perturbation strategy distant:
Input Tokens (Rows) vs. Output Tokens (Columns)


,The (0),clock (1),shows (2),(3),9 (4),: (5),47 (6),PM (7),. (8)
The (0),0.216258,0.005785,0.001539,0.001589,0.000149,0.000088,0.000135,0.000155,0.005681
clock (1),0.059312,0.100215,0.003406,0.001585,0.000279,0.000046,0.001556,0.000514,0.000375
shows (2),0.100506,0.005279,0.014283,0.008331,0.000237,0.000098,0.000504,0.000200,0.010792
9 (3),0.120247,0.004997,0.011580,0.069102,0.952396,0.000277,0.967580,0.000195,0.004938
: (4),0.120247,0.004997,0.011580,0.069102,0.952396,0.000277,0.967580,0.000195,0.004938
47 (5),0.120247,0.004997,0.011580,0.069102,0.952396,0.000277,0.967580,0.000195,0.004938
PM (6),0.071975,0.017281,0.002717,0.000139,0.000227,0.000100,0.000094,0.999861,0.992498
. (7),0.071975,0.017281,0.002717,0.000139,0.000227,0.000100,0.000094,0.999861,0.992498
What (8),0.115970,0.005619,0.154163,0.016166,0.004919,0.000001,0.005443,0.002075,0.174994
time (9),0.122300,0.003139,0.009951,0.048819,0.950056,0.000044,0.000167,0.953905,0.992498


Attribution matrix for token_displacement with perturbation strategy distant:
Input Tokens (Rows) vs. Output Tokens (Columns)


,The (0),clock (1),shows (2),(3),9 (4),: (5),47 (6),PM (7),. (8)
The (0),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
clock (1),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
shows (2),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9 (3),0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,6.000000,0.000000,0.000000
: (4),0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,6.000000,0.000000,0.000000
47 (5),0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,6.000000,0.000000,0.000000
PM (6),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,20.000000
. (7),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,20.000000
What (8),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
time (9),0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,20.000000


In [23]:
logger.print_attribution_matrix(1, "cosine")

Attribution matrix for cosine with perturbation strategy distant:
Input Tokens (Rows) vs. Output Tokens (Columns)


,The (0),clock (1),shows (2),9 (3),: (4),47 (5),PM (6),. (7)
The (0),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
clock (1),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
shows (2),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
9 (3),0.000000,-0.000000,-0.000000,0.289177,0.000000,0.274936,0.000000,0.000000
: (4),0.000000,-0.000000,-0.000000,0.289177,0.000000,0.274936,0.000000,0.000000
47 (5),0.000000,-0.000000,-0.000000,0.289177,0.000000,0.274936,0.000000,0.000000
PM (6),0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.821500,1.000000
. (7),0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.821500,1.000000
What (8),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
time (9),0.000000,-0.000000,0.000000,0.488054,0.000000,-0.000000,0.821500,1.000000


In [24]:
logger.print_attribution_matrix(2, attribution_strategy="cosine")
logger.print_attribution_matrix(2, attribution_strategy="prob_diff")

Attribution matrix for cosine with perturbation strategy distant:
Input Tokens (Rows) vs. Output Tokens (Columns)


,The (0),clock (1),shows (2),9 (3),: (4),47 (5),PM (6),. (7)
The (0),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
clock (1),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
shows (2),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
9 (3),0.000000,-0.000000,-0.000000,0.289177,0.000000,0.274936,0.000000,0.000000
: (4),0.000000,-0.000000,-0.000000,0.289177,0.000000,0.274936,0.000000,0.000000
47 (5),0.000000,-0.000000,-0.000000,0.289177,0.000000,0.274936,0.000000,0.000000
PM (6),0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.821500,1.000000
. (7),0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.821500,1.000000
What (8),0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
time (9),0.000000,-0.000000,0.000000,0.488054,0.000000,-0.000000,0.821500,1.000000


Attribution matrix for prob_diff with perturbation strategy distant:
Input Tokens (Rows) vs. Output Tokens (Columns)


,The (0),clock (1),shows (2),(3),9 (4),: (5),47 (6),PM (7),. (8)
The (0),0.064448,0.007869,0.003085,0.002989,0.000108,0.000128,0.000118,0.000054,0.007132
clock (1),0.136458,0.033954,0.003810,0.002736,0.000285,0.000101,0.001414,0.000432,0.000383
shows (2),0.151668,0.006804,0.009290,0.006151,0.000079,0.000141,0.000733,0.000125,0.016142
9 (3),0.191758,0.008194,0.016663,0.040286,0.951041,0.000315,0.958258,0.000138,0.001367
: (4),0.191758,0.008194,0.016663,0.040286,0.951041,0.000315,0.958258,0.000138,0.001367
47 (5),0.191758,0.008194,0.016663,0.040286,0.951041,0.000315,0.958258,0.000138,0.001367
PM (6),0.171069,0.000117,0.001073,0.000121,0.000113,0.000139,0.000016,0.999858,0.994526
. (7),0.171069,0.000117,0.001073,0.000121,0.000113,0.000139,0.000016,0.999858,0.994526
What (8),0.198133,0.007814,0.148431,0.018417,0.003066,0.000065,0.003120,0.001621,0.126054
time (9),0.196840,0.005341,0.009895,0.077305,0.884765,0.000052,0.000132,0.925403,0.994526


In [19]:
logger.print_attribution_matrix(3, attribution_strategy="cosine")
logger.print_attribution_matrix(3, attribution_strategy="prob_diff")

Attribution matrix for cosine with perturbation strategy distant:
Input Tokens (Rows) vs. Output Tokens (Columns)


,The (0),building (1),is (2),132 (3),meters (4),tall (5),. (6)
The (0),0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000
building (1),0.226659,0.705013,0.520012,0.806286,0.855425,0.756068,0.466330
is (2),0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000
132 (3),0.226659,0.770341,0.621099,0.807823,0.785745,0.767478,0.480238
met (4),0.226659,0.705013,0.520012,0.788010,0.798989,0.771913,0.412676
ers (5),0.226659,0.705013,0.520012,0.788010,0.798989,0.771913,0.412676
tall (6),0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000
. (7),0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000
How (8),0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000
tall (9),0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000


Attribution matrix for prob_diff with perturbation strategy distant:
Input Tokens (Rows) vs. Output Tokens (Columns)


,The (0),building (1),is (2),(3),132 (4),meters (5),tall (6),. (7)
The (0),0.018736,0.000113,0.000309,0.000079,0.000053,0.000006,0.000121,0.000022
building (1),0.777744,0.999023,0.999883,0.999955,0.999962,0.999989,0.999822,0.999864
is (2),0.015364,0.000713,0.008316,0.000166,0.000016,0.000007,0.000146,0.000301
132 (3),0.985685,0.999023,0.999883,0.999893,0.999962,0.999989,0.999822,0.999864
met (4),0.963465,0.999023,0.999883,0.999942,0.999962,0.999989,0.999822,0.999864
ers (5),0.963465,0.999023,0.999883,0.999942,0.999962,0.999989,0.999822,0.999864
tall (6),0.057443,0.000039,0.000001,0.001233,0.000211,0.000013,0.001011,0.000231
. (7),0.057443,0.000039,0.000001,0.001233,0.000211,0.000013,0.001011,0.000231
How (8),0.017349,0.013034,0.006253,0.351449,0.024528,0.000197,0.034164,0.228418
tall (9),0.028325,0.024104,0.002190,0.004585,0.000051,0.000016,0.000100,0.001761
